# Key recovery for loop randomization

This notebook focuses on the task of key recovery using different side-channel analysis methods.
All methods focus on _first order leakage_ as the loop randomization (i.e. hiding) does not require higher order analysis.

## Methods

A good overview and explanation of existing side-channel analysis methods can be found in [Power Side-Channel Attack Analysis: A Review of 20 Years of Study for the Layman](https://www.mdpi.com/2410-387X/4/2/15).

### CPA

A very common method is CPA.
Is uses the [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) to reveal a _linear dependency_ between a predicted leakage and the actual traces.

The predictive value is free to choose: An entire key byte, it's hamming weight, a single bit, ...

## T-Test

The concept of side-channel analysis using [Welch's T-Test](https://en.wikipedia.org/wiki/Welch%27s_t-test) is to divide the recorded traces into two groups by a given _selection function_. The t-statistic gives a (unit-less) number which is high when the two sets "are different".

In classical DPA (e.g. [Power Side-Channel Attack Analysis: A Review of 20 Years of Study for the Layman](https://www.mdpi.com/2410-387X/4/2/15) 4.1) a _partition function_ is used to distinguish a set of traces into two distinct partitions.
Primarily the partition function focuses on a single bit of a computed value inside a cryptographic algorithm where a known part (mostly plaintext or ciphertext) collides with the secret key.
By partitioning with different key hypothesis one can "guess" the correct one: It is the partition where the two groups are most different from each other.

Kocher et al. proposed to use the "DoM (Difference of Means)" to compute a "difference" between two partitions.
The T-Test in contrast provides a better success rate [Empirical Comparison of Side Channel Analysis Distinguishers on DES in Hardware](https://www.esat.kuleuven.be/cosic/publications/article-1250.pdf).


Recovering the key from an AES SBOX lookup the following partition function can be used:

```python
sbox(plaintext[i] ^ guess) & (1 << b)
```

Considering the `i`-th byte and the `b`-th bit the total amount of partitions is $16 \cdot 8 \cdot 256 = 32768$.

## Signal-to-Noise Ratio (SNR)

> Signal-to-noise ratios are commonly used in electrical engineering and signal processing. An SNR is tbe ratio between tbe signal and tbe noise component of a measurement. The general definition of an SNR in a digital environment is given in (4.9).
> $$ SNR = \frac{Var(Signal)}{Var(Noise)} $$

(Mangard S., Oswald E., Popp T. Power Analysis Attacks.. Revealing the Secrets of Smart Cards (Advances in Information Security) (2007) 4.3.2)

For precise computation refer to [Advanced side-channel Measurement and Testing](https://hss-opus.ub.ruhr-uni-bochum.de/opus4/frontdoor/deliver/index/docId/8024/file/diss.pdf) (Formula 5.5).

$$ \mathrm{SNR} = \frac{
    \mathrm{Var}_{\forall k \in \mathcal S}(\mathbf{\bar x}_k)
}{
    \mathrm{E}_{\forall k \in \mathcal S}\big(\mathrm{Var}_{\forall i}(\mathbf{x}_{k,i})\big)
} $$

Similar to T-Test we can partition the set of traces into multiple subsets.
But, in contrast we don't have to focus on a single bit.
The entire byte or it's Hamming Weight can be considered.


## Comparison

A widely used criteria for comparing algorithms for key recovery is _ranking_.
Obviously, a method is "better" then another if the key can be revealed requiring less traces.
Hence, as we know the correct key (byte) a priori, we can check at which position in the ranking is occurs.

### Fictive example

We are guessing 256 possible key bytes. 
After including 10 traces into the first key recovery algorithm we see the correct key is only at position 200. 
For algorithm two it is a position 150.
This does not (yet) mean necessarily that algorithm one is better than two!

But, after 1000 traces we see that the correct key is also the best guess of algorithm one but only the 3rd best guess of algorithm two.
We realize that algorithm one is predominant to algorithm two.

## CW Plain Fixedkey

First of all we throw the weakest implementation into the ring and compare the results.